In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "lab_data/Video_Games_5.json"
N = 10


In [5]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    import re
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt", 'useful']
    rows = []

    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        useful = d["helpful"]
        rows.append((uid, iid, review, rating, dt, useful))
        
        
    return pd.DataFrame(rows, columns=cols)

df = get_data_frame()

In [57]:
df.head()

,uid,iid,review,rating,dt,useful
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000,"[8, 12]"
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400,"[0, 0]"
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600,"[0, 0]"
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400,"[7, 10]"
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600,"[2, 2]"


In [ ]:
training_df, test_df = split_df_by_dt(df, p=0.8)
del df

In [58]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [66]:
def prepare_items_matrix(df, max_features=None, del_texts=True):
    from scipy.sparse import csr_matrix
    from scipy.sparse import vstack
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    id_dict = {} # сщщтветствие id объекта и перечня строк, относящихся к этому объекту в df
    cntr = 0 # счетчик строк
    texts = [] # массив текстов
    items_list = [] # массив для промежуточного хранения векторов объектов
    
    for iid in df.iid.unique():
        dummy = []
        for rewiew in df.loc[df.iid == iid].review:
            dummy.append(cntr)
            texts.append(review)
            cntr+=1
        id_dict[iid] = dummy
    
    # создаем векторизатор
    tfvect = TfidfVectorizer(stop_words='english', max_features=max_features).fit(texts)
    voc_len = len(tfvect.vocabulary_)
        
    for iid, rows in id_dict.items():
        item_vec = csr_matrix((1,voc_len))
        for i in rows:
            review = texts[i]
            vect = tfidfvectorizer.transform(review)
            item_vect += vect
        item_vect = item_vect/len(rows)
        items_list.append(item_vect)
        iid_to_row[iid] = iid_to_row.setdefault(iid, len(iid_to_row))
    
    items_matrix = vstack(items_list)
    del items_list
    if del_texts:
        del texts
    
    return items_matrix, iid_to_row